In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-70b-8192")

## Runnable PassThrough

* It does not do anything to the input data.
* Let's see it in a very simple example: chain with just RunnablePassThrough() will output the original input without any modifications.

In [3]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()

In [4]:
chain.invoke("Lokesh")

'Lokesh'

## Runnable Lambda

* To use custom function inside a LCEL chain, we need to wrap it up with Runnable Lambda.
* Let's define a very simple function to create Russian lastnames:

In [5]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

In [6]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

In [7]:
chain.invoke("Lokesh")

'Lokeshovich'

## Runnable Parallel

* We will use RunnableParallel() for running tasks in parallel.
* This is probably the most important and most useful Runnable from LangChain.
* In the following chain, RunnableParallel is going to run these two tasks in parallel:
    1. operation_a will use RunnablePassthrough.
    2. operation_b will use RunnableLambda with a russian_lastname function.

In [8]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [9]:
chain.invoke("Lokesh")

{'operation_a': 'Lokesh', 'operation_b': 'Lokeshovich'}

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [11]:
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

In [12]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough(),
    }
) | prompt | model | output_parser

In [13]:
chain.invoke({
    "name1": "Messi",
    "name": "Abram",
})

"Roman Abramovich, the Russian billionaire and former owner of Chelsea FC!\n\nHere's a curious fact about him:\n\nDid you know that Roman Abramovich was once a street trader who sold plastic ducks and rubber toys on the streets of Moscow? Yes, you read that right! Before he became one of the richest men in the world, Abramovich started his business career by selling toys and souvenirs on the streets of Moscow. He was just 16 years old at the time. Who would have thought that this humble beginning would eventually lead him to become a billionaire with a net worth of over $15 billion?"

### Let's see a more advanced use of Runnable Parallel

In [14]:
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq  

In [15]:
vectorstore = FAISS.from_texts(
    ["dswithbappy focuses on providing content related to Data Science, AI, ML, NLP, DL, CV, Python Programming etc. in English"], 
    embedding= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device': 'cpu'})
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatGroq(model="llama3-70b-8192")

retriever_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

retriever_chain.invoke("What is dswithbappy?")

C:\Users\Lokesh\AppData\Local\Temp\ipykernel_17952\719447652.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device': 'cpu'})
c:\Users\Lokesh\anaconda3\envs\llmapp\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


'Based on the provided context, dswithbappy is a entity that focuses on providing content related to various technical topics such as Data Science, AI, ML, NLP, DL, CV, and Python Programming, all in the English language.'

### Alternate way to use the runnable parallel

In [16]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq 

vectorstore = FAISS.from_texts(
    ["dswithbappy focuses on providing content related to Data Science, AI, ML, NLP, DL, CV, Python Programming etc. in English"], 
    embedding= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device': 'cpu'})
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatGroq(model="llama3-70b-8192")

#Whenever you see Curly braces{} inside chain, then it will be act as RunnableParallel
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "What is dswithbappy?", "language": "Hindi"})

'दस्विथबैप्पी डेटा साइंस, एआई, एमएल, एनएलपी, डीएल, सीवी, पायथन प्रोग्रामिंग आदि से संबंधित सामग्री प्रदान करने पर केंद्रित है।'

In [18]:
chain.invoke({"question": "What is dswithbappy?", "language": "Marathi"})

'ड्सविथबप्पी हे डेटा सायन्स, एआय, एमएल, एनएलपी, डीएल, सीव्ही, पायथन प्रोग्रामिंग इ. संबंधित इंग्रजी मधील सामग्री प्रदान करण्यावर केंद्रित आहे.'

In [19]:
chain.invoke({"question": "What is dswithbappy?", "language": "Spanish"})

'Según el contexto, dswithbappy se enfoca en proporcionar contenido relacionado con Ciencia de Datos, IA, ML, NLP, DL, CV y programación en Python en inglés.'